Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-data-dependency-steps.png)

# Azure Machine Learning Pipelines with Data Dependency
In this notebook, we will see how we can build a pipeline with implicit data dependency.

## Prerequisites and Azure Machine Learning Basics
If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the [configuration Notebook](https://aka.ms/pl-config) first if you haven't. This sets you up with a working config file that has information on your workspace, subscription id, etc. 

### Azure Machine Learning and Pipeline SDK-specific Imports

In [1]:
import azureml.core
from azureml.core import Workspace, Experiment, Datastore
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.widgets import RunDetails

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

from azureml.data.data_reference import DataReference
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep
print("Pipeline SDK-specific imports completed")

SDK version: 1.0.85
Pipeline SDK-specific imports completed


### Initialize Workspace

Initialize a [workspace](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace(class%29) object from persisted configuration.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# Default datastore (Azure blob storage)
# def_blob_store = ws.get_default_datastore()
def_blob_store = Datastore(ws, "workspaceblobstore")
print("Blobstore's name: {}".format(def_blob_store.name))

avadevitsmlsvc
RG-ITSMLTeam-Dev
westus2
ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e
Blobstore's name: workspaceblobstore


### Source Directory
The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

In [3]:
# source directory
source_directory = 'data_dependency_run_train'
    
print('Sample scripts will be created in {} directory.'.format(source_directory))

Sample scripts will be created in data_dependency_run_train directory.


### Required data and script files for the the tutorial
Sample files required to finish this tutorial are already copied to the project folder specified above. Even though the .py provided in the samples don't have much "ML work," as a data scientist, you will work on this extensively as part of your work. To complete this tutorial, the contents of these files are not very important. The one-line files are for demostration purpose only.

### Compute Targets
See the list of Compute Targets on the workspace.

In [4]:
cts = ws.compute_targets
for ct in cts:
    print(ct)

test-steve
test-alieu
adf
CU-Int
nema-sobhani
deal-add-Estimat
deal-add-Est-big
deal-master
ret-master-roll
deal-deals-sj-te
deal-deals-s-big
ret-testitspymlp
ret-testits-roll
ret-newtoken
ret-newtoke-roll
ret-handlenewsta
ret-handlen-roll
deal-hist-pred-o
deal-histpredict
deal-histpre-big
deal-fixpreds
deal-fixpred-big
deal-deal-incorr
deal-deal-in-big
deal-fix-gatekee
deal-fix-gat-big
deal-gatekeeper
deal-gatekee-big
deal-dw-debug-an
deal-dw-debu-big
deal-optimizeCos
deal-optimiz-big
ret-optimizeCosm
ret-optimiz-roll
ret-doslasheswor
ret-doslash-roll
deal-predict-clo
ret-pinpackageve
ret-pinpack-roll
ret-theSDKstrike
ret-theSDKs-roll
deal-deals-sj-ex
ret-holdoutpredi
ret-holdout-roll
ret-datadrift
ret-datadri-roll
deal-drop-stage
deal-drop-st-big
deal-make-dims-c
deal-make-di-big
deal-add-regress
deal-add-reg-big
gpu-cluster
ret-cleanthonali
ret-cleanth-roll
ret-metricsfix
ret-metrics-roll
deal-clean-alieu
deal-clean-a-big
deal-hackathon-s
deal-hackath-big
ret-hackathonscr
ret-hackat

#### Retrieve or create an Aml compute
Azure Machine Learning Compute is a service for provisioning and managing clusters of Azure virtual machines for running machine learning workloads. Let's get the default Aml Compute in the current workspace. We will then run the training script on this compute target.

In [5]:
from azureml.core.compute_target import ComputeTargetException

aml_compute_target = "cpu-cluster"
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("found existing compute target.")
except ComputeTargetException:
    print("creating new compute target")
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                min_nodes = 1, 
                                                                max_nodes = 4)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Aml Compute attached")


creating new compute target
Creating
Succeeded..................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
Aml Compute attached


In [6]:
# For a more detailed view of current Azure Machine Learning Compute status, use get_status()
# example: un-comment the following line.
# print(aml_compute.get_status().serialize())

**Wait for this call to finish before proceeding (you will see the asterisk turning to a number).**

Now that you have created the compute target, let's see what the workspace's compute_targets() function returns. You should now see one entry named 'amlcompute' of type AmlCompute.

## Building Pipeline Steps with Inputs and Outputs
As mentioned earlier, a step in the pipeline can take data as input. This data can be a data source that lives in one of the accessible data locations, or intermediate data produced by a previous step in the pipeline.

### Datasources
Datasource is represented by **[DataReference](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.data_reference.datareference?view=azure-ml-py)** object and points to data that lives in or is accessible from Datastore. DataReference could be a pointer to a file or a directory.

In [7]:
# Reference the data uploaded to blob storage using DataReference
# Assign the datasource to blob_input_data variable

# DataReference(datastore, 
#               data_reference_name=None, 
#               path_on_datastore=None, 
#               mode='mount', 
#               path_on_compute=None, 
#               overwrite=False)

blob_input_data = DataReference(
    datastore=def_blob_store,
    data_reference_name="test_data",
    path_on_datastore="20newsgroups/20news.pkl")
print("DataReference object created")

DataReference object created


### Intermediate/Output Data
Intermediate data (or output of a Step) is represented by **[PipelineData](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-core/azureml.pipeline.core.pipelinedata?view=azure-ml-py)** object. PipelineData can be produced by one step and consumed in another step by providing the PipelineData object as an output of one step and the input of one or more steps.

#### Constructing PipelineData
- **name:** [*Required*] Name of the data item within the pipeline graph
- **datastore_name:** Name of the Datastore to write this output to
- **output_name:** Name of the output
- **output_mode:** Specifies "upload" or "mount" modes for producing output (default: mount)
- **output_path_on_compute:** For "upload" mode, the path to which the module writes this output during execution
- **output_overwrite:** Flag to overwrite pre-existing data

In [8]:
# Define intermediate data using PipelineData
# Syntax

# PipelineData(name, 
#              datastore=None, 
#              output_name=None, 
#              output_mode='mount', 
#              output_path_on_compute=None, 
#              output_overwrite=None, 
#              data_type=None, 
#              is_directory=None)

# Naming the intermediate data as processed_data1 and assigning it to the variable processed_data1.
processed_data1 = PipelineData("processed_data1",datastore=def_blob_store)
print("PipelineData object created")

PipelineData object created


### Pipelines steps using datasources and intermediate data
Machine learning pipelines can have many steps and these steps could use or reuse datasources and intermediate data. Here's how we construct such a pipeline:

#### Define a Step that consumes a datasource and produces intermediate data.
In this step, we define a step that consumes a datasource and produces intermediate data.

**Open `train.py` in the local machine and examine the arguments, inputs, and outputs for the script. That will give you a good sense of why the script argument names used below are important.** 

#### Specify conda dependencies and a base docker image through a RunConfiguration

This step uses a docker image and scikit-learn, use a [**RunConfiguration**](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.runconfiguration?view=azure-ml-py) to specify these requirements and use when creating the PythonScriptStep. 

In [9]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

# create a new runconfig object
run_config = RunConfiguration()

# enable Docker 
run_config.environment.docker.enabled = True

# set Docker base image to the default CPU-based image
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

# use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_config.environment.python.user_managed_dependencies = False

# specify CondaDependencies obj
run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

In [46]:
# step4 consumes the datasource (Datareference) in the previous step
# and produces processed_data1
source_directory = 'data_dependency_run_train'
trainStep = PythonScriptStep(
    script_name="train.py", 
    arguments=["--input_data", blob_input_data, "--output_train", processed_data1,
    "--test", "will this run again?"
    ],
    inputs=[blob_input_data],
    outputs=[processed_data1],
    compute_target=aml_compute, 
    source_directory=source_directory,
    runconfig=run_config,
    allow_reuse=True
)
print("trainStep created")

trainStep created


#### Define a Step that consumes intermediate data and produces intermediate data
In this step, we define a step that consumes an intermediate data and produces intermediate data.

**Open `extract.py` in the local machine and examine the arguments, inputs, and outputs for the script. That will give you a good sense of why the script argument names used below are important.** 

In [47]:
# step5 to use the intermediate data produced by step4
# This step also produces an output processed_data2
processed_data2 = PipelineData("processed_data2", datastore=def_blob_store)
source_directory = "data_dependency_run_extract"

extractStep = PythonScriptStep(
    script_name="extract.py",
    arguments=["--input_extract", processed_data1, "--output_extract", processed_data2],
    inputs=[processed_data1],
    outputs=[processed_data2],
    compute_target=aml_compute, 
    source_directory=source_directory)
print("extractStep created")

extractStep created


#### Define a Step that consumes intermediate data and existing data and produces intermediate data
In this step, we define a step that consumes multiple data types and produces intermediate data.

This step uses the output generated from the previous step as well as existing data on a DataStore. The location of the existing data is specified using a [**PipelineParameter**](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-core/azureml.pipeline.core.pipelineparameter?view=azure-ml-py) and a [**DataPath**](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.datapath.datapath?view=azure-ml-py). Using a PipelineParameter enables easy modification of the data location when the Pipeline is published and resubmitted.

**Open `compare.py` in the local machine and examine the arguments, inputs, and outputs for the script. That will give you a good sense of why the script argument names used below are important.**

In [48]:
# Reference the data uploaded to blob storage using a PipelineParameter and a DataPath
from azureml.pipeline.core import PipelineParameter
from azureml.data.datapath import DataPath, DataPathComputeBinding

datapath = DataPath(datastore=def_blob_store, path_on_datastore='20newsgroups/20news.pkl')
datapath_param = PipelineParameter(name="compare_data", default_value=datapath)
data_parameter1 = (datapath_param, DataPathComputeBinding(mode='mount'))

In [49]:
# Now define the compare step which takes two inputs and produces an output
processed_data3 = PipelineData("processed_data3", datastore=def_blob_store)
source_directory = "data_dependency_run_compare"

compareStep = PythonScriptStep(
    script_name="compare.py",
    arguments=["--compare_data1", data_parameter1, "--compare_data2", processed_data2, "--output_compare", processed_data3],
    inputs=[data_parameter1, processed_data2],
    outputs=[processed_data3],    
    compute_target=aml_compute, 
    source_directory=source_directory)
print("compareStep created")

compareStep created


#### Build the pipeline

In [50]:
pipeline1 = Pipeline(workspace=ws, steps=[compareStep])
print ("Pipeline is built")

Pipeline is built


In [51]:
pipeline_run1 = Experiment(ws, 'Data_dependency').submit(pipeline1)
print("Pipeline is submitted for execution")

Created step compare.py [d7163749][88cf741a-ef1b-473d-9ef6-914dc197632f], (This step will run and generate new outputs)
Created step extract.py [2e06287a][eec910d2-d13a-4a05-acc4-1c5d1b82e0c5], (This step is eligible to reuse a previous run's output)
Created step train.py [6af8d595][7a57e7bb-5105-4610-b129-0f46450d318d], (This step will run and generate new outputs)
Created data reference workspaceblobstore_9d25b107 for StepId [2862b4b7][68653175-4ca2-43ca-b38d-a1c7348fd331], (Consumers of this data will generate new runs.)
Using data reference test_data for StepId [a47508df][e412565a-205a-4edd-b57e-57ac47b14acd], (Consumers of this data are eligible to reuse prior runs.)
Submitted PipelineRun 5279d32d-5c63-48fb-b5eb-4961cec9d6f5
Link to Azure Machine Learning studio: https://ml.azure.com/experiments/Data_dependency/runs/5279d32d-5c63-48fb-b5eb-4961cec9d6f5?wsid=/subscriptions/ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e/resourcegroups/RG-ITSMLTeam-Dev/workspaces/avadevitsmlsvc
Pipeline is sub

In [52]:
# RunDetails(pipeline_run1).show()

#### Wait for pipeline run to complete

In [53]:
pipeline_run1.wait_for_completion(show_output=True)

ummary
StepRun( extract.py ) Status: Finished
{'runId': '74456ff9-4d26-48f9-a758-3bb867081700', 'target': 'cpu-cluster', 'status': 'Completed', 'startTimeUtc': '2020-02-06T20:06:02.739034Z', 'endTimeUtc': '2020-02-06T20:06:50.245479Z', 'properties': {'azureml.runsource': 'azureml.StepRun', 'ContentSnapshotId': '41dfb837-6c74-480d-8ade-8f88fbee11c2', 'StepType': 'PythonScriptStep', 'ComputeTargetType': 'AmlCompute', 'azureml.pipelinerunid': '5279d32d-5c63-48fb-b5eb-4961cec9d6f5', '_azureml.ComputeTargetType': 'amlcompute', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'runDefinition': {'script': 'extract.py', 'useAbsolutePath': False, 'arguments': ['--input_extract', '$AZUREML_DATAREFERENCE_processed_data1', '--output_extract', '$AZUREML_DATAREFERENCE_processed_data2'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'cpu-cluster', 'dataReferences': {'processe

'Finished'

### See Outputs

See where outputs of each pipeline step are located on your datastore.

***Wait for pipeline run to complete, to make sure all the outputs are ready***

In [ ]:
# Get Steps
for step in pipeline_run1.get_steps():
    print("Outputs of step " + step.name)
    
    # Get a dictionary of StepRunOutputs with the output name as the key 
    output_dict = step.get_outputs()
    
    for name, output in output_dict.items():
        
        output_reference = output.get_port_data_reference() # Get output port data reference
        print("\tname: " + name)
        print("\tdatastore: " + output_reference.datastore_name)
        print("\tpath on datastore: " + output_reference.path_on_datastore)

### Download Outputs

We can download the output of any step to our local machine using the SDK.

In [ ]:
# Retrieve the step runs by name 'train.py'
train_step = pipeline_run1.find_step_run('train.py')

if train_step:
    train_step_obj = train_step[0] # since we have only one step by name 'train.py'
    train_step_obj.get_output_data('processed_data1').download("./outputs") # download the output to current directory

# Next: Publishing the Pipeline and calling it from the REST endpoint
See this [notebook](https://aka.ms/pl-pub-rep) to understand how the pipeline is published and you can call the REST endpoint to run the pipeline.